In [2]:
import pandas as pd
import pandasql as ps
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder

In [3]:

df = pd.read_csv("../references/EXTR_RPSale.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
q1 = ("""
SELECT * 
FROM df
WHERE DocumentDate like "%2019"
""")
df2019 =ps.sqldf(q1)

In [5]:
df2019.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,3027422,213043,120,12/20/2019,560000,20191226000848,,,,,...,11,6,3,N,N,N,N,1,8,
1,3002257,940652,630,07/22/2019,435000,20190730001339,,,,,...,11,6,3,N,N,N,N,1,8,
2,2999169,919715,200,07/08/2019,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
3,2980648,797320,2320,03/27/2019,540000,,,,,,...,3,6,3,N,N,N,N,1,8,
4,3013394,663990,178,10/02/2019,0,20191002000875,,,,,...,3,6,15,N,N,N,N,19,8,18 31 51 52


In [8]:
df2019.to_csv('../references/df2019',index=False)